

# 🧱 Setting Up Product and Customer Dimension Tables in Unity Catalog

This setup creates two dimension tables in Unity Catalog under `company.unit`. These tables will enrich transactional data by adding product and customer context — a foundational step for building a retail data pipeline.

This code sets up two Delta Lake dimension tables — product_dim and customer_dim — under the Unity Catalog path company.unit. The product_dim table stores product metadata such as ID, category, and price, while the customer_dim table contains customer details like name and region. Sample data is inserted into both tables for enrichment purposes. These tables are foundational for joining with transactional data to support analytics and reporting.


In [0]:

-- Use the correct catalog and schema
USE CATALOG company;
USE SCHEMA unit;

-- Product Dimension Table
CREATE OR REPLACE TABLE product_dim (
  product_id INT,
  product_name STRING,
  category STRING,
  price DOUBLE
) USING DELTA;

INSERT INTO product_dim VALUES
  (101, 'Phone', 'Electronics', 500.0),
  (102, 'Laptop', 'Electronics', 1200.0),
  (103, 'Shirt', 'Clothing', 40.0),
  (104, 'Shoes', 'Clothing', 80.0);

-- Customer Dimension Table
CREATE OR REPLACE TABLE customer_dim (
  customer_id INT,
  customer_name STRING,
  region STRING
) USING DELTA;

INSERT INTO customer_dim VALUES
  (1, 'Alice', 'North'),
  (2, 'Bob', 'West'),
  (3, 'Carol', 'East');


This code sets the working context to the company catalog and unit schema in Unity Catalog. It then creates a managed volume named sales_input_volume, which is a governed storage location for file-based data. The volume is used to store streaming input files, such as JSON records for sales transactions. This enables secure, trackable ingestion into Databricks pipelines using Autoloader or file-based reads.

In [0]:
USE CATALOG company;
USE SCHEMA unit;

CREATE VOLUME IF NOT EXISTS sales_input_volume
COMMENT 'Volume for streaming sales input JSON files';

This command creates (or replaces) a Delta Lake table named sales_raw under the current Unity Catalog catalog and schema. The table is designed to store raw sales transaction data with fields like timestamp, transaction ID, customer ID, product ID, and quantity. It acts as a bronze layer table in the data pipeline, typically populated using streaming ingestion from files stored in a volume. This table serves as the starting point for further enrichment and analysis.

In [0]:

CREATE OR REPLACE TABLE sales_raw (
  transaction_time TIMESTAMP,
  transaction_id INT,
  customer_id INT,
  product_id INT,
  quantity INT
) USING DELTA;


This code sets up a streaming DataFrame using Databricks Autoloader (cloudFiles) to ingest JSON files from the Unity Catalog volume path /Volumes/company/unit/sales_input_volume/. It defines the schema for incoming sales transaction data with fields like timestamp, transaction ID, customer ID, product ID, and quantity. The data is read as a real-time stream, enabling continuous or micro-batch processing for downstream analytics and transformations.

In [0]:
%python

from pyspark.sql.functions import col, window, expr, sum as spark_sum, count

sales_stream_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .schema("transaction_time TIMESTAMP, transaction_id INT, customer_id INT, product_id INT, quantity INT")
    .load("/Volumes/company/unit/sales_input_volume/")
)


This code reads static dimension tables product_dim and customer_dim from Unity Catalog and joins them with the streaming sales data (sales_stream_df). It enriches each transaction with product and customer details. A new column spend is computed by multiplying quantity and price, representing the total transaction value. This prepares the enriched dataset for downstream analytics like customer spend tracking or category-level aggregations.

In [0]:
%python

product_df = spark.read.table("company.unit.product_dim")
customer_df = spark.read.table("company.unit.customer_dim")

enriched_sales = sales_stream_df \
    .join(product_df, "product_id", "left") \
    .join(customer_df, "customer_id", "left") \
    .withColumn("spend", col("quantity") * col("price"))


This code sets the active catalog to company and schema to unit within Unity Catalog. It then creates a managed volume named chk, which will be used to store checkpoint data for streaming queries. Checkpoints are essential for maintaining streaming state and ensuring exactly-once processing. This setup allows Databricks Structured Streaming to recover from failures and resume from the last successful state.

In [0]:
USE CATALOG company;
USE SCHEMA unit;

CREATE VOLUME IF NOT EXISTS chk
COMMENT 'Volume for streaming sales input JSON files';

This code calculates lifetime spending per customer by aggregating the spend column from the enriched sales stream. It writes the aggregated result to a Delta table named company.unit.customer_spending. The stream uses trigger(availableNow=True), which processes all available data once and then stops. A checkpoint is stored in the chk volume to ensure recoverability and incremental processing in future runs.

In [0]:
%python
customer_spending = enriched_sales.groupBy("customer_id").agg(
    spark_sum("spend").alias("lifetime_spent")
)

customer_spending.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .trigger(availableNow=True) \
    .option("checkpointLocation", "/Volumes/company/unit/chk/customer_spending") \
    .toTable("company.unit.customer_spending")


#Simulate

This code uses dbutils.fs.put() to write a sample JSON file named sample.json into the Unity Catalog volume sales_input_volume. The file contains a single sales transaction record with fields like timestamp, customer ID, product ID, and quantity. This JSON file simulates streaming input data for testing the Autoloader pipeline. The overwrite=True flag ensures the file is replaced if it already exists.


In [0]:
%python

dbutils.fs.put("/Volumes/company/unit/sales_input_volume/sample.json", """
{
  "transaction_time": "2024-06-06T10:00:00",
  "transaction_id": 1001,
  "customer_id": 1,
  "product_id": 101,
  "quantity": 1
}
""", overwrite=True)


#Validate

In [0]:
%sql

select * from company.unit.customer_spending


#Adding row

This command writes another sample JSON file named sample1.json into the Unity Catalog volume sales_input_volume. It represents a sales transaction where customer 2 purchases 3 units of product 102 at a specific timestamp. This file serves as additional input data for the streaming pipeline. The overwrite=True flag ensures the file is replaced if it already exists, supporting repeated testing.

In [0]:
%python

dbutils.fs.put("/Volumes/company/unit/sales_input_volume/sample1.json", """
{
  "transaction_time": "2024-06-06T10:00:00",
  "transaction_id": 1002,
  "customer_id": 2,
  "product_id": 102,
  "quantity": 3
}
""", overwrite=True)

Second Test

#Test and Validate

In [0]:
%python
customer_spending = enriched_sales.groupBy("customer_id").agg(
    spark_sum("spend").alias("lifetime_spent")
)

customer_spending.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .trigger(availableNow=True) \
    .option("checkpointLocation", "/Volumes/company/unit/chk/customer_spending") \
    .toTable("company.unit.customer_spending")

    %sql

select * from company.unit.customer_spending